In [10]:
from math import *
import numpy as np

def traverse(tree,string=''):
    if tree.num_children == 1:
        string += (tree.op)
        string += ('('+str(tree.coeff)+'*'+traverse(tree.left)+')')
    elif tree.num_children==2:
        string += ('('+traverse(tree.left)+')')
        string += (tree.op)
        string += ('('+traverse(tree.right)+')')
    else:
        string += '('+str(tree.coeff) + '*' + str(tree.value)+')'
    return string

In [11]:
# intermediate_operands = ['sin', 'cos', 'exp', 'np.log', '+', '*']
# unary_operands = ['sin', 'cos', 'np.log', 'exp']
unary_operands = ['sin', 'cos', 'np.log']
binary_operands = ['*', '/', '+', '-', '**']
# terminal_operands = ["var1", "1"]
terminal_operands = ["1"]
coeff_max = 3



In [70]:
class PrintTree:
    """source: https://ebadahmadzadeh.wordpress.com/2014/02/26/how-to-print-a-dictionary-based-tree-in-python/"""
    def printTree(self, tree, d = 0):
        if (tree.num_children == 0):
            print("\t" * d, tree.index, tree.value)
        else:
            print( "\t" * d, tree.index, tree.name, tree.op)
            for i in [tree.left, tree.right]:
                if i != None:
                    self.printTree(i, d+1)
#                 else:
#                     print ("\t" * d, str('(') + "f" + str(')'))

In [71]:
for i in range(100):
    n1 = node(None, 1, "root")
    p = PrintTree()
    generate_tree(n1, [0.2, 0.4, 0.4], 0.08)
#     eval(traverse(n1))
    try:
        eval(traverse(n1))
    except:
        print(n1)
        print(traverse(n1))
        break
#     print(n1)
#     print(traverse(n1))
#     print(p.printTree(n1))
#     print(n1)

NameError: name 'generate_tree' is not defined

In [138]:
class bin_tree():
    def __init__(self):
        self.root = node("root")
        self.num_nodes = 0
        self.node_list = []
        
    def generate_tree(self, node, r_coeffs, delta):
        "generates tree recursively from a starter node"
        self.num_nodes += 1
        node.index = self.num_nodes
        self.node_list.append(node)
        r = np.random.rand()
        c = r_coeffs
        #decrease p(double branch) while
        #increase p(no branch)
        c[0] += delta
        c[1] -= delta/2
        c[2] -= delta/2
        if r < r_coeffs[0]:
            node.coeff = np.random.uniform(coeff_max)
            node.value = np.random.choice(terminal_operands)
        elif r > r_coeffs[0] and r < r_coeffs[1]:
            node.init_left()
            node.op = np.random.choice(unary_operands)
            node.coeff = np.random.uniform(coeff_max)
            self.generate_tree(node.left, c, delta)
        else:
            node.init_left()
            node.init_right()
            node.op = np.random.choice(binary_operands)
            node.coeff = 1

            self.generate_tree(node.left, c, delta) 
            self.generate_tree(node.right, c, delta)
    
            
    def get_root(self):
        return self.root
            
    def __repr__(self):
        return traverse(self.root)

In [139]:
class node():
    
#     def __init__(self, index, name, parent=None):
#         self.name = name
#         self.num_children=0
#         self.index=index
#         self._left=None
#         self._right=None
#         self._op=None
#         self.coeff=None
#         self._value = None
#         self.parent = parent
        
#     def init_left(self, index):
#         left_node = node(index, "left", parent=self)
#         self.left = left_node
#         self.num_children += 1
        
#     def init_right(self, index):
#         right_node = node(index, "right", parent=self)
#         self.right = right_node
#         self.num_children += 1 


    def __init__(self, name, parent=None):
        self.name = name
        self.num_children=0
        self.index=None
        self._left=None
        self._right=None
        self._op=None
        self.coeff=None
        self._value = None
        self.parent = parent
        
    def init_left(self):
        left_node = node("left", parent=self)
        self.left = left_node
        self.num_children += 1
        
    def init_right(self):
        right_node = node("right", parent=self)
        self.right = right_node
        self.num_children += 1 
    
    @property    
    def left(self):
        return self._left
    
    @property    
    def right(self):
        return self._right
    
    @property    
    def op(self):
        return self._op
    
    @property    
    def value(self):
        return self._value
    
    @value.setter
    def value(self, val):
        self._value = val
    
    @left.setter
    def left(self, node):
        self._left = node
    
    @right.setter
    def right(self, node):
        self._right = node

    @op.setter
    def op(self, op):
        self._op = op
    
    def __repr__(self):
        if self.num_children > 0:
            return "(left: {}, op: {}, right: {})".format(
                self.left, self.op, self.right)
        return "{}*{}".format(self.coeff, self.value)

In [140]:
# start = node(None, 1)
# start.init_left()
# start.init_right()
# print(start)
a = node(1, "asdf")
a.init_left(2)
a.init_right(3)

TypeError: init_left() takes 1 positional argument but 2 were given

In [146]:
a = bin_tree()

a.generate_tree(a.get_root(), [0.2, 0.4, 0.4], 0.04)

eval(traverse(a.get_root()))

(1.700809975025402e+19-1.4362249447999616e+21j)

In [147]:
p.printTree(a.get_root())

 1 root **
	 2 left *
		 3 left **
			 4 left **
				 5 left **
					 6 left -
						 7 left +
							 8 1
							 9 1
						 10 right *
							 11 1
							 12 right *
								 13 left +
									 14 1
									 15 1
								 16 1
					 17 1
				 18 1
			 19 1
		 20 1
	 21 1


In [149]:
len(a.node_list)

21

In [135]:
def traverse_index(tree, a):
    tree.index = a
    if tree.num_children == 1:
        tree.index = a
        a += 1
        traverse_index(tree.left, a)
    elif tree.num_children==2:
        tree.index = a
        a += 1
        traverse_index(tree.left, a)
        a += 1
        traverse_index(tree.right, a)

        

In [128]:
# g = bin_tree()

# g.generate_tree(g.get_root(), [0.2, 0.4, 0.4], 0.04)

# a=1
s=1
traverse_index(a.get_root(), s)
p.printTree(a.get_root())

 1 1
